In [1]:
import tensorflow as tf

In [2]:
# Check if GPU is available and set TensorFlow to use it
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("GPUs available:", physical_devices)
    try:
        for device in physical_devices:
            tf.config.experimental.set_memory_growth(device, True)
        print("TensorFlow is set to use GPU.")
    except Exception as e:
        print("Could not set memory growth:", e)
else:
    print("No GPU found. Running on CPU.")

GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow is set to use GPU.


In [3]:

from tensorflow import keras    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

In [4]:
train_path = './dataset/train'
valid_path = './dataset/val'
test_path = './dataset/test'

breeds = [
    "Holstein_cow", "Jersey_cow", "Angus_cow", "Brahman_cow", "Hereford_cow",
    "Simmental_cow", "Limousin_cow", "Guernsey_cow", "Charolais_cow", "Ayrshire_cow"
]

from tensorflow.keras.applications.densenet import preprocess_input

train_batches = ImageDataGenerator(preprocessing_function=preprocess_input)\
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=breeds, batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=preprocess_input)\
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=breeds, batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=preprocess_input)\
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=breeds, batch_size=10, shuffle=False)

Found 2802 images belonging to 10 classes.
Found 1069 images belonging to 10 classes.
Found 1083 images belonging to 10 classes.


In [5]:
assert train_batches.n >= 1000
assert valid_batches.n >= 300
assert test_batches.n >= 150
assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 10
assert train_batches.batch_size == valid_batches.batch_size == test_batches.batch_size == 10
assert train_batches.class_indices == valid_batches.class_indices == test_batches.class_indices

imgs , labels = next(train_batches)
print("Shape of the image batch: ", imgs.shape)
print("Shape of the label batch: ", labels.shape)

Shape of the image batch:  (10, 224, 224, 3)
Shape of the label batch:  (10, 10)


In [6]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize = (20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr,axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
    plotImages(imgs)
print(labels)

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [7]:
model = Sequential([
    Conv2D(filters=32,kernel_size=(3,3),activation='relu', padding='same',input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2,2),strides=2),
    Conv2D(filters=64, kernel_size = (3,3), activation = 'relu', padding = 'same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=10, activation='softmax'),
])

In [8]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')
import time
from tensorflow.keras.applications import ResNet50, DenseNet121, MobileNetV3Large
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1660, compute capability 7.5


In [9]:
def build_model(base_model_class, input_shape, num_classes):
    base_model = base_model_class(weights='imagenet', include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    optimizer = Adam(learning_rate=0.001)  
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Helper function to calculate mAP
def mean_average_precision(y_true, y_pred):
    # y_true and y_pred should be numpy arrays
    # This is a simple implementation for multiclass, multi-label
    import sklearn.metrics
    return sklearn.metrics.average_precision_score(y_true, y_pred, average='macro')

input_shape = (224, 224, 3)
num_classes = len(breeds)
epochs = 50
results = {}

In [10]:
print("Training DenseNet121...")
densenet121_model = build_model(DenseNet121, input_shape, num_classes)
start_time = time.time()
history = densenet121_model.fit(
    train_batches,
    validation_data=valid_batches,
    epochs=epochs,
    verbose=2,
)
training_time = time.time() - start_time

Training DenseNet121...
Epoch 1/50
281/281 - 243s - loss: 1.9434 - accuracy: 0.3451 - val_loss: 7.3778 - val_accuracy: 0.1431 - 243s/epoch - 864ms/step
Epoch 2/50
281/281 - 221s - loss: 1.6850 - accuracy: 0.4293 - val_loss: 3.3056 - val_accuracy: 0.1581 - 221s/epoch - 788ms/step
Epoch 3/50
281/281 - 221s - loss: 1.5488 - accuracy: 0.4939 - val_loss: 3.6805 - val_accuracy: 0.3358 - 221s/epoch - 786ms/step
Epoch 4/50
281/281 - 223s - loss: 1.3313 - accuracy: 0.5678 - val_loss: 3.9324 - val_accuracy: 0.2778 - 223s/epoch - 795ms/step
Epoch 5/50
281/281 - 225s - loss: 1.1613 - accuracy: 0.6067 - val_loss: 1.7244 - val_accuracy: 0.5042 - 225s/epoch - 799ms/step
Epoch 6/50
281/281 - 226s - loss: 1.0500 - accuracy: 0.6610 - val_loss: 1.4096 - val_accuracy: 0.5313 - 226s/epoch - 803ms/step
Epoch 7/50
281/281 - 224s - loss: 0.9299 - accuracy: 0.7020 - val_loss: 1.6941 - val_accuracy: 0.5921 - 224s/epoch - 798ms/step
Epoch 8/50
281/281 - 226s - loss: 0.8256 - accuracy: 0.7323 - val_loss: 0.8110 -

In [11]:
val_imgs, val_labels = next(valid_batches)
val_preds = densenet121_model.predict(val_imgs)
acc = np.mean(np.argmax(val_preds, axis=1) == np.argmax(val_labels, axis=1))
mAP = mean_average_precision(val_labels, val_preds)
results['DenseNet121'] = {
    'training_time_sec': training_time,
    'val_accuracy': acc,
    'val_mAP': mAP,
    'history': history,
    'val_imgs': val_imgs,
    'val_labels': val_labels,
    'val_preds': val_preds
}

1/1 [==============================] - 2s 2s/step


c:\Users\Mansal\anaconda3\envs\Mansal\lib\site-packages\sklearn\metrics\_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\Mansal\anaconda3\envs\Mansal\lib\site-packages\sklearn\metrics\_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
c:\Users\Mansal\anaconda3\envs\Mansal\lib\site-packages\sklearn\metrics\_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [12]:
with open('results_densenet121.pkl', 'wb') as file:
    pickle.dump(results, file)

INFO:tensorflow:Assets written to: ram://9e81bad2-4cae-4156-a20a-447411e52e8f/assets


INFO:tensorflow:Assets written to: ram://9e81bad2-4cae-4156-a20a-447411e52e8f/assets


In [13]:
for model_name, res in results.items():
    print(f"Model: {model_name}")
    print(f"  Training Time: {res['training_time_sec']:.2f} seconds")
    print(f"  Validation Accuracy: {res['val_accuracy']:.4f}")
    print(f"  Validation mAP: {res['val_mAP']:.4f}")
    print("-" * 40)

Model: DenseNet121
  Training Time: 7533.18 seconds
  Validation Accuracy: 1.0000
  Validation mAP: 0.7000
----------------------------------------
